# ECG Classification using ResNet 1D CNN

## Load Required Libraries

In [5]:
%%capture
!pip install wfdb

In [8]:
import os
import wfdb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

ModuleNotFoundError: No module named 'wfdb'

## Data Loading and Preprocessing

In [7]:
# Set the download path
download_path = 'dataset'

# Create the download folder if it doesn't exist
if not os.path.exists(download_path):
    os.makedirs(download_path)

    # Download the MIT-BIH Arrhythmia Database
    wfdb.dl_database('mitdb', download_path=download_path)

NameError: name 'wfdb' is not defined

In [ ]:
# Load the dataset

In [ ]:
# Preprocess the data
signals = record.p_signal
labels = annotation.symbol

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(signals, labels, test_size=0.2, random_state=42)

# Train and test your machine learning model using any algorithm of your choice